https://www.kaggle.com/berkayalan/vaccinations-in-united-states
us_vaccinations.csv

# The main goal of this dashboard is to display the pogress of vaccinations in the 52 United states

## in this notebook I will eplore the best way to display the data. Later I will make a django app where I am aiming to display the following views:
* A heatmap of the united states to keep track of the amount of vacinnations in each state
* A linechart for each state filtered by state and date to diaplay the change in vaccinations
* A bar chart of pie chart to display the amount of people vaccinted compared to the total population per state

*Seaborn: Size of the state == box --> vaccination vs total population filling up the box

# Imports

In [30]:
import pandas as pd
import sqlite3 as sql
from bokeh.io import push_notebook, output_notebook 
from bokeh.plotting import figure,show
from bokeh.embed import components
from bokeh.models import ColumnDataSource, HoverTool,Legend
from bokeh.palettes import Magma,RdYlGn
from bokeh.transform import factor_cmap

In [31]:
output_notebook() #to see notebook in the app

Loading BokehJS ...

## Opening dataset that contains vaccine information per state

In [32]:
db = '../dash/project_2.db'
conn = sql.connect(db)

In [33]:
vacc = pd.read_sql("SELECT * from vacc", conn, index_col='index')
vacc.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
index,,,,,,,,,,,,,,
0,12.01.2021,Alabama,78134.0,377025.0,70861.0,0.15,21551.0,7270.0,16438.0,25385.0,0.0,0.0,0.0,0.207
1,13.01.2021,Alabama,84040.0,378975.0,74792.0,0.19,25934.0,9245.0,19360.0,26846.0,5906.0,5906.0,1205.0,0.222
2,14.01.2021,Alabama,92300.0,435350.0,80480.0,0.19,32143.0,9245.0,23377.0,32356.0,8260.0,7083.0,1445.0,0.212
3,15.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,8267.0,7478.0,1525.0,0.226
4,16.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,7557.0,7498.0,1529.0,0.226


In [34]:
vacc[vacc['location'] == 'Vermont']

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
index,,,,,,,,,,,,,,
9371,12.01.2021,Vermont,25757.0,64925.0,1542739.0,36.66,41365.00,1175281.0,48.12,44296.00,0.00,0.0,0.0,0.397
9372,13.01.2021,Vermont,29571.0,69800.0,1542739.0,36.66,27120.00,1175281.0,48.12,43770.00,3814.00,3814.0,6112.0,0.424
9373,14.01.2021,Vermont,29571.0,69800.0,26603.0,36.66,27120.00,1175281.0,46113.00,43770.00,0.00,1907.0,3056.0,0.424
9374,15.01.2021,Vermont,32470.0,74000.0,28355.0,0.66,44232.00,4100.0,19815.00,31717.00,2899.00,2238.0,3587.0,0.439
9375,16.01.2021,Vermont,32470.0,74000.0,28355.0,0.66,44232.00,4100.0,19815.00,31717.00,2027.75,2185.0,3502.0,0.439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9530,20.06.2021,Vermont,838245.0,912630.0,456145.0,64.13,134.34,400190.0,73.10,146.26,2318.00,3273.0,5245.0,0.918
9531,21.06.2021,Vermont,839933.0,912630.0,456656.0,64.34,134.61,401475.0,73.18,146.26,1688.00,2794.0,4478.0,0.920
9532,22.06.2021,Vermont,840948.0,912630.0,457072.0,64.45,134.77,402130.0,73.25,146.26,1015.00,2136.0,3423.0,0.921


In [35]:
total = pd.read_sql("SELECT * from total", conn, index_col='index')
total = total.sort_values(by='doses_needed', ascending=False)

In [86]:
total.head()

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop,total_vaccinations_thousand
index,,,,,,,,,
4,CA,36.778261,-119.417932,California,39538223,40975374,79076446,51,79076
44,TX,31.968599,-99.901813,Texas,29145505,24341655,58291010,41,58291
9,FL,27.664827,-81.515754,Florida,21538187,20053457,43076374,46,43076
34,NY,43.299428,-74.217933,New York State,20201249,20841312,40402498,51,40402
38,PA,41.203322,-77.194525,Pennsylvania,13011844,13390121,26023688,51,26023


each state: Range(smallest 1/2million, largest 40million). Each dot will represent a state. 
    We want to see if some states are clustering between each other. To see if we cna find that
    states with bigger population have less percentage and vise versa
    x1: numpy array of 50 total num of doses
    y2: numpy array of 50 elements containing the percent vaccinated

In [87]:
doses = total['total_vaccinations_thousand'].values
percentage = total['perc_daily_vacc_pop'].values

In [94]:
doses

array([79076, 58291, 43076, 40402, 26023, 25625, 23598, 21423, 20878,
       20154, 18577, 17262, 15410, 14303, 14059, 13821, 13571, 12354,
       12309, 11787, 11547, 11412, 10236, 10048,  9315,  9011,  8474,
        7918,  7211,  6571,  6543,  6380,  6209,  6023,  5922,  5875,
        4235,  3923,  3678,  3587,  2910,  2755,  2724,  2194,  2168,
        1979,  1773,  1558,  1466,  1379,  1286,  1153])

In [95]:
percentage

array([51, 41, 46, 51, 51, 48, 43, 37, 40, 45, 52, 51, 53, 43, 60, 35, 39,
       53, 39, 48, 50, 49, 39, 32, 34, 42, 51, 38, 58, 45, 41, 45, 42, 37,
       31, 40, 52, 44, 34, 36, 56, 52, 56, 55, 40, 49, 43, 38, 41, 59, 63,
       34])

In [88]:
print(len(total['location']))

52


In [89]:
total['total_vaccinations_thousand']= total['doses_needed']//1000

In [90]:
color_lst = [x for x in Magma[4]]*13

In [99]:
def scatter_plot():
#     total = read_total()
    total = pd.read_sql("SELECT * from total", conn, index_col='index')
    total = total.sort_values(by='doses_needed', ascending=False)
    total['total_vaccinations_thousand']= total['doses_needed']//1000
    doses = total['total_vaccinations_thousand'].values
    percentage = total['perc_daily_vacc_pop'].values

    colors = [x for x in Magma[4]]*13

    p = figure(title = "State Percentage and Doses distribution")
    p.xaxis.axis_label = 'Percentage of People Fully Vaccinated (%)'
    p.yaxis.axis_label = 'total Doses Needed (thousand)'

    p.circle(percentage, doses,
             color=colors, fill_alpha=0.2, size=10)

    hover = HoverTool()
    p.add_tools(hover)

    return p
    
scatter_plot()

In [16]:
from bokeh.plotting import figure, output_file, show
from bokeh.sampledata.iris import flowers

colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
print(len(flowers['species']))
colors = [colormap[x] for x in flowers['species']]
print(len(colors))

p = figure(title = "Iris Morphology")
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

print(len(flowers["petal_length"]))
p.circle(flowers["petal_length"], flowers["petal_width"],
         color=colors, fill_alpha=0.2, size=10)


show(p)

150
150
150
